<a href="https://colab.research.google.com/github/vitthal-bhandari/Homophobia-Transphobia-Detection/blob/master/Ablations/Data_Ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Initialize environment

In [ ]:
#check gpu usage
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [ ]:
#check ram usage
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
!pip install datasets
!pip install transformers
!pip install pyspellchecker
!pip install emoji --upgrade

     |████████████████████████████████| 325 kB 4.4 MB/s 
     |████████████████████████████████| 77 kB 7.2 MB/s 
     |████████████████████████████████| 212 kB 78.4 MB/s 
     |████████████████████████████████| 136 kB 70.1 MB/s 
     |████████████████████████████████| 1.1 MB 61.9 MB/s 
     |████████████████████████████████| 127 kB 75.1 MB/s 
     |████████████████████████████████| 144 kB 64.3 MB/s 
     |████████████████████████████████| 271 kB 89.7 MB/s 
     |████████████████████████████████| 94 kB 2.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 13.5 MB/s 

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from huggingface_hub import notebook_login
from spellchecker import SpellChecker
from datasets import load_dataset
import tensorflow as tf
import pandas as pd
import numpy as np
import emoji

# 2. Read Data

## 2.1 Read English data

In [ ]:
eng_train_dataset = load_dataset("csv", data_files="eng_3_train.tsv", sep="\t")
eng_train = eng_train_dataset["train"]
eng_dev_dataset = load_dataset("csv", data_files="eng_3_dev.tsv", sep="\t")
eng_dev = eng_dev_dataset["train"]
eng_test_dataset = load_dataset("csv", data_files="eng_3_test.tsv", sep="\t")
eng_test = eng_test_dataset["train"]
print(len(eng_train), len(eng_dev), len(eng_test))

Using custom data configuration default-31cb70369d28572b
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-31cb70369d28572b/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-fca620da9955125c
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-fca620da9955125c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-053122befc584ac6
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-053122befc584ac6/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

3164 792 999


In [ ]:
eng_train = eng_train.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'])
eng_train = eng_train.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_train = eng_train.class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-31cb70369d28572b/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-36174075c57bd4af.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-31cb70369d28572b/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-ed75665133cec481.arrow


In [ ]:
eng_dev = eng_dev.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9'])
eng_dev = eng_dev.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_dev = eng_dev.class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-fca620da9955125c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-be417f6b92f5836e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-fca620da9955125c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-0b54abca55581e1b.arrow


In [ ]:
eng_test = eng_test.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_test = eng_test.rename_column(original_column_name= 'text                        ', new_column_name= 'text')
eng_test = eng_test.class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-053122befc584ac6/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-7361bf98ddec36e5.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-053122befc584ac6/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-adbeff8f4746c0b2.arrow


In [ ]:
# remove existing NA rows in test set
eng_test = eng_test.filter(lambda example, indice: indice < 990, with_indices=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-053122befc584ac6/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-ac345f8d2f7c3b09.arrow


In [ ]:
eng_train, eng_dev, eng_test

(Dataset({
     features: ['label', 'text'],
     num_rows: 3164
 }), Dataset({
     features: ['label', 'text'],
     num_rows: 792
 }), Dataset({
     features: ['label', 'text'],
     num_rows: 990
 }))

**Converting Datasets to Datafrane**

In [ ]:
eng_train.set_format(type="pandas")
eng_dev.set_format(type="pandas")
eng_test.set_format(type="pandas")

df_eng_train = eng_train[:]
df_eng_dev = eng_dev[:]
df_eng_test = eng_test[:]

df_eng_train

,label,text
0,1,"I support her, very smart ponnu"
1,0,priyadharshini kannan same gender attraction ...
2,1,Bro u name and phone number (or)mobile number ...
3,1,experience Thaks bro I love you so much bro
4,1,world is becoming bad day by day....
...,...,...
3159,1,That Akka is really good soul
3160,1,Good madam.if i get any oppertunity in future ...
3161,1,judgment is based on scientific facts
3162,1,Respect to allll🙏🙏🙏


## 2.2 Read Tamil data

In [ ]:
tam_train_dataset = load_dataset("csv", data_files="tam_3_train.tsv", sep="\t")
tam_train = tam_train_dataset["train"]
tam_dev_dataset = load_dataset("csv", data_files="tam_3_dev.tsv", sep="\t")
tam_dev = tam_dev_dataset["train"]
tam_test_dataset = load_dataset("csv", data_files="tam_3_test.tsv", sep="\t")
tam_test = tam_test_dataset["train"]
print(len(tam_train), len(tam_dev), len(tam_test))

Using custom data configuration default-3564ca8db5a1cf9a
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-3564ca8db5a1cf9a/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-e8723891884620fc
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-e8723891884620fc/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-ebc8885dbb8851bc
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-ebc8885dbb8851bc/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

2662 666 833


In [ ]:
tam_train, tam_dev, tam_test

(Dataset({
     features: ['category', 'text', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'],
     num_rows: 2662
 }), Dataset({
     features: ['category', 'text', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17'],
     num_rows: 666
 }), Dataset({
     features: ['category', 'text', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed:

In [ ]:
tam_train = tam_train.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'
                                       , 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'])
tam_train = tam_train.rename_column(original_column_name= 'category', new_column_name= 'label')
tam_train = tam_train.class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-3564ca8db5a1cf9a/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-0a8043dac1e52dc0.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-3564ca8db5a1cf9a/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-3670d02953d6593e.arrow


In [ ]:
tam_dev = tam_dev.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'
                                       , 'Unnamed: 16', 'Unnamed: 17'])
tam_dev = tam_dev.rename_column(original_column_name= 'category', new_column_name= 'label')
tam_dev = tam_dev.class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-e8723891884620fc/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-3e51b7bcdb873f9a.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-e8723891884620fc/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-a7bab61f99ac7846.arrow


In [ ]:
tam_test = tam_test.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'
                                       , 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'])
tam_test = tam_test.rename_column(original_column_name= 'category', new_column_name= 'label')
tam_test = tam_test.class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-ebc8885dbb8851bc/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-6f9da747be70002e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-ebc8885dbb8851bc/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-6daff5ecf5aa299d.arrow


In [ ]:
tam_train, tam_dev, tam_test

(Dataset({
     features: ['label', 'text'],
     num_rows: 2662
 }), Dataset({
     features: ['label', 'text'],
     num_rows: 666
 }), Dataset({
     features: ['label', 'text'],
     num_rows: 833
 }))

**Converting Datasets to Datafrane**

In [ ]:
tam_train.set_format(type="pandas")
tam_dev.set_format(type="pandas")
tam_test.set_format(type="pandas")

df_tam_train = tam_train[:]
df_tam_dev = tam_dev[:]
df_tam_test = tam_test[:]

df_tam_train

,label,text
0,2,அர்த்தநாரீஸ்வரர் ஆணுமானவர் பெண்ணுமானவர். இது ஒ...
1,1,section377 | ஓரினச்சேர்க்கை நல்லதா கெட்டதா மக்...
2,2,இருக்கிற நாலு பேர் சேர்ந்து அந்த பொறம்போக்கு ச...
3,1,Shanth sha உடலுறவு இனப்பெருக்கத்திற்கு மட்டும...
4,1,ஜீவநதி ப்ரியன் hi
...,...,...
2657,0,நீங்கள் என்னதான் சட்டம் கொண்டுவந்தாலும் இது தவ...
2658,1,Ayy Ar இதய துடிப்பில் உனது நினைவலைகள். ஊன் உற...
2659,1,இவர்கள் இவ்வளவு பண்ணிவிட்டு எங்களை சமுதாயம் மத...
2660,1,ப்ரோ அவங்களுக்கு.திருநங்கைகள் தான் இவங்க.கட்டப...


## 2.3 Read Code-Mixed data

In [ ]:
eng_tam_train_dataset = load_dataset("csv", data_files="eng_tam_3_train.tsv", sep="\t")
eng_tam_train = eng_tam_train_dataset["train"]
eng_tam_dev_dataset = load_dataset("csv", data_files="eng_tam_3_dev.tsv", sep="\t")
eng_tam_dev = eng_tam_dev_dataset["train"]
eng_tam_test_dataset = load_dataset("csv", data_files="eng_tam_3_test.tsv", sep="\t")
eng_tam_test = eng_tam_test_dataset["train"]
print(len(eng_tam_train), len(eng_tam_dev), len(eng_tam_test))

Using custom data configuration default-a3624288f194dd88
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-a3624288f194dd88/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-7dec3aa9d2b539d7
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-7dec3aa9d2b539d7/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-3aa9e7bdad12a7a9
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-3aa9e7bdad12a7a9/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

3861 966 1207


In [ ]:
eng_tam_train, eng_tam_dev, eng_tam_test

(Dataset({
     features: ['category', 'text', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27'],
     num_rows: 3861
 }), Dataset({
     features: ['category', 'text', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19'],
     num_rows: 966
 }), Dataset({
     features: ['category', 'text', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed:

In [ ]:
eng_tam_train = eng_tam_train.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'
                                       , 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'
                                       , 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27'])
eng_tam_train = eng_tam_train.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_tam_train = eng_tam_train.class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-a3624288f194dd88/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-6a1cfc8102f73f83.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-a3624288f194dd88/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-1f32a3a957664d96.arrow


In [ ]:
eng_tam_dev = eng_tam_dev.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'
                                       , 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19'])
eng_tam_dev = eng_tam_dev.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_tam_dev = eng_tam_dev.class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-7dec3aa9d2b539d7/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-c191ccbac03a6481.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-7dec3aa9d2b539d7/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-575d2b3d12a29a27.arrow


In [ ]:
eng_tam_test = eng_tam_test.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'
                                       , 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19'])
eng_tam_test = eng_tam_test.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_tam_test = eng_tam_test.class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-3aa9e7bdad12a7a9/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-884d3f626270d366.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-3aa9e7bdad12a7a9/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-3c9e56f27a17ec72.arrow


In [ ]:
eng_tam_train, eng_tam_dev, eng_tam_test

(Dataset({
     features: ['label', 'text'],
     num_rows: 3861
 }), Dataset({
     features: ['label', 'text'],
     num_rows: 966
 }), Dataset({
     features: ['label', 'text'],
     num_rows: 1207
 }))

**Converting Datasets to Datafrane**

In [ ]:
eng_tam_train.set_format(type="pandas")
eng_tam_dev.set_format(type="pandas")
eng_tam_test.set_format(type="pandas")

df_eng_tam_train = eng_tam_train[:]
df_eng_tam_dev = eng_tam_dev[:]
df_eng_tam_test = eng_tam_test[:]

df_eng_tam_train

,label,text
0,0,@Kadhayalla Nijam thaa gay Punda Apo ne confi...
1,1,love you too Nanba
2,1,Semma super 👍👍👍👍
3,1,Ava ka elom kadavuluka
4,1,Bro gay persons kalyanam panninaa kozhanda pet...
...,...,...
3856,1,Antha akka romba caring
3857,1,@Dan_Lei Nii Pooi Vidurayaa Unakku Yaarum v...
3858,1,Frnds neenga antha yedathula a irrunthaalum am...
3859,1,Looking pretty 🤩


## 2.4 Drop empty rows

In [ ]:
df_eng_train = df_eng_train.dropna()
df_eng_dev = df_eng_dev.dropna()
df_eng_test = df_eng_test.dropna()

df_eng_train = df_eng_train.reset_index(drop=True)
df_eng_dev = df_eng_dev.reset_index(drop=True)
df_eng_test = df_eng_test.reset_index(drop=True)

In [ ]:
df_tam_train = df_tam_train.dropna()
df_tam_dev = df_tam_dev.dropna()
df_tam_test = df_tam_test.dropna()

df_tam_train = df_tam_train.reset_index(drop=True)
df_tam_dev = df_tam_dev.reset_index(drop=True)
df_tam_test = df_tam_test.reset_index(drop=True)

In [ ]:
df_eng_tam_train = df_eng_tam_train.dropna()
df_eng_tam_dev = df_eng_tam_dev.dropna()
df_eng_tam_test = df_eng_tam_test.dropna()

df_eng_tam_train = df_eng_tam_train.reset_index(drop=True)
df_eng_tam_dev = df_eng_tam_dev.reset_index(drop=True)
df_eng_tam_test = df_eng_tam_test.reset_index(drop=True)

## 2.5 Save non-preprocessed and non-augmented data

In [ ]:
df_eng_train.to_csv('eng_train_nonprepro_nonaug.csv', index=False)
df_eng_dev.to_csv('eng_dev_nonprepro_nonaug.csv', index=False)
df_eng_test.to_csv('eng_test_nonprepro_nonaug.csv', index=False)

df_tam_train.to_csv('tam_train_nonprepro_nonaug.csv', index=False)
df_tam_dev.to_csv('tam_dev_nonprepro_nonaug.csv', index=False)
df_tam_test.to_csv('tam_test_nonprepro_nonaug.csv', index=False)

df_eng_tam_train.to_csv('eng_tam_train_nonprepro_nonaug.csv', index=False)
df_eng_tam_dev.to_csv('eng_tam_dev_nonprepro_nonaug.csv', index=False)
df_eng_tam_test.to_csv('eng_tam_test_nonprepro_nonaug.csv', index=False)

## 2.6 Remove emojis

In [ ]:
df_eng_train['text'] = df_eng_train['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
df_eng_dev['text'] = df_eng_dev['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
df_eng_test['text'] = df_eng_test['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [ ]:
df_tam_train['text'] = df_tam_train['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
df_tam_dev['text'] = df_tam_dev['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
df_tam_test['text'] = df_tam_test['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [ ]:
df_eng_tam_train['text'] = df_eng_tam_train['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
df_eng_tam_dev['text'] = df_eng_tam_dev['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
df_eng_tam_test['text'] = df_eng_tam_test['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

## 2.7 Remove unnecessary puncutation

In [ ]:
#keras tokenizer

train_text_tok = Tokenizer(split=' ', lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
train_text_tok.fit_on_texts(df_eng_train['text'])

dev_text_tok = Tokenizer(split=' ', lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
dev_text_tok.fit_on_texts(df_eng_dev['text'])

test_text_tok = Tokenizer(split=' ', lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
test_text_tok.fit_on_texts(df_eng_test['text'])

In [ ]:
train_tok = train_text_tok.texts_to_sequences(df_eng_train['text'])

df_eng_train['text'] = df_eng_train.apply(lambda row: train_text_tok.sequences_to_texts([train_tok[row.name]])[0], axis=1)

In [ ]:
dev_tok = dev_text_tok.texts_to_sequences(df_eng_dev['text'])

df_eng_dev['text'] = df_eng_dev.apply(lambda row: dev_text_tok.sequences_to_texts([dev_tok[row.name]])[0], axis=1)

In [ ]:
test_tok = test_text_tok.texts_to_sequences(df_eng_test['text'])

df_eng_test['text'] = df_eng_test.apply(lambda row: test_text_tok.sequences_to_texts([test_tok[row.name]])[0], axis=1)

## 2.8 Run spell check

In [ ]:
spell = SpellChecker()

spell.word_frequency.load_words(['lgbt', 'lgbtq', 'lgbtqia+', 'gay', 'lesbian', 'transgender', 'trans'])

In [ ]:
def spell_check(row):
  correct_sent = []
  misspelled = spell.unknown(row.split())
  for word in row.split():
    if word in misspelled:
      correct_sent.append(spell.correction(word))
    else:
      correct_sent.append(word)

  return " ".join(correct_sent)

In [ ]:
df_eng_train['text'] = df_eng_train['text'].apply(spell_check)
df_eng_dev['text'] = df_eng_dev['text'].apply(spell_check)
df_eng_test['text'] = df_eng_test['text'].apply(spell_check)

**Review training dataframe once again**

In [ ]:
df_eng_train

,label,text
0,1,i support her very smart ponzu
1,0,priyadharshini kanan same gender attraction wi...
2,1,bro u name and phone number or mobile number p...
3,1,experience thanks bro i love you so much bro
4,1,world is becoming bad day by day
...,...,...
3157,1,that akka is really good soul
3158,1,good madam if i get any opportunity in future ...
3159,1,judgment is based on scientific facts
3160,1,respect to alll folded hands folded hands fold...


## 2.9 Save preprocessed but non-augmented data

In [ ]:
df_eng_train.to_csv('eng_train_prepro_nonaug.csv', index=False)
df_eng_dev.to_csv('eng_dev_prepro_nonaug.csv', index=False)
df_eng_test.to_csv('eng_test_prepro_nonaug.csv', index=False)

df_tam_train.to_csv('tam_train_prepro_nonaug.csv', index=False)
df_tam_dev.to_csv('tam_dev_prepro_nonaug.csv', index=False)
df_tam_test.to_csv('tam_test_prepro_nonaug.csv', index=False)

df_eng_tam_train.to_csv('eng_tam_train_prepro_nonaug.csv', index=False)
df_eng_tam_dev.to_csv('eng_tam_dev_prepro_nonaug.csv', index=False)
df_eng_tam_test.to_csv('eng_tam_test_prepro_nonaug.csv', index=False)

## 2.10 Reset Dataset format

In [ ]:
eng_train.reset_format()
eng_dev.reset_format()
eng_test.reset_format()

In [ ]:
tam_train.reset_format()
tam_dev.reset_format()
tam_test.reset_format()

In [ ]:
eng_tam_train.reset_format()
eng_tam_dev.reset_format()
eng_tam_test.reset_format()

# 3. EDA

**Saving dataset in a csv format classwise separately**

In [ ]:
df_homophobic=df_eng_train[df_eng_train.label == 0]
df_homophobic.head()

,label,text
1,0,priyadharshini kannan same gender attraction ...
22,0,Really excellent movie....I feel that no one a...
31,0,Madam read bible
94,0,it's due to abnormalities present sexual dimor...
96,0,i think u too gay....thats y u telling with t...


In [ ]:
df_transphobic=df_eng_train[df_eng_train.label == 2]
df_transphobic.head()

,label,text
401,2,Sex the 9.
1090,2,Magalakshmi Mukunthan Ella transgalayum konnud...
1340,2,Today also one transgender scolded me for not ...
1418,2,Rajeswari Rajagopalan ; Sister....last week fo...
2330,2,Hey seriously I thought She was a Transgender


**Applying EDA to homophobia and transphobia classes**

In [ ]:
import nltk

nltk.download('wordnet')
!git clone https://github.com/jasonwei20/eda_nlp.git
%cd eda_nlp/code

!ls

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
fatal: destination path 'eda_nlp' already exists and is not an empty directory.
/content/eda_nlp/code
augment.py	    eda.py		 homophobic.txt  transphobic.txt
eda_homophobic.txt  eda_transphobic.txt  __pycache__


In [ ]:
df_homophobic.to_csv('homophobic.txt', sep="\t", header=False, index=False)
df_transphobic.to_csv('transphobic.txt', sep="\t", header=False, index=False)

In [ ]:
! python augment.py --input=transphobic.txt --num_aug=32

generated augmented sentences with eda for transphobic.txt to eda_transphobic.txt with num_aug=32


In [ ]:
! python augment.py --input=homophobic.txt --num_aug=16

generated augmented sentences with eda for homophobic.txt to eda_homophobic.txt with num_aug=16


**Combine augmented data with existing data**

In [ ]:
df_homophobic_aug=pd.read_csv('eda_homophobic.txt', sep="\t", names=["label", "text"])
df_transphobic_aug=pd.read_csv('eda_transphobic.txt', sep="\t", names=["label", "text"])

In [ ]:
df_homophobic_aug

,label,text
0,0,with kannan same gender attraction with sex fi...
1,0,priyadharshini kannan same gender attraction w...
2,0,priyadharshini kannan gender attraction with s...
3,0,chum priyadharshini kannan same gender attract...
4,0,priyadharshini sex same gender attraction an s...
...,...,...
2664,0,people others mindset seems that there people ...
2665,0,anu lavanya it seems people likely to theft yo...
2666,0,to lavanya it seems that some people likely to...
2667,0,anu lavanya it seems that some people likely t...


In [ ]:
df_ally=df_eng_train[df_eng_train.label == 1]
df_ally.describe()

,label
count,2999.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [ ]:
df_ally_shuffled=df_ally.sample(frac=1).reset_index(drop=True)
df_ally_shuffled_subset=df_ally_shuffled.iloc[:1500]

In [ ]:
df_ally_shuffled_subset

,label,text
0,1,Habeeb Aslam oh please ... there is nothing ca...
1,1,Madan bro do a video about 13 secret society f...
2,1,Bala kumar she is right we don't need Ur opin...
3,1,Bible says a lot more. But I don't think anyon...
4,1,I really hope she rethought her decisions. Th...
...,...,...
1495,1,She is so beautiful. I am ur biggest fan. I ha...
1496,1,black sheep I am the Director of that film
1497,1,Why do these LGBT couples have to adopt childr...
1498,1,rajeshs who said about pay? I was commenting...


In [ ]:
frames = [df_ally_shuffled_subset, df_homophobic, df_transphobic, df_homophobic_aug, df_transphobic_aug]
df_aug = pd.concat(frames)

In [ ]:
# re shuffling dataframe

df_aug_shuffled = df_aug.sample(frac=1).reset_index(drop=True)
df_aug_shuffled

,label,text
0,1,Semme speech ....superb
1,0,dont support lgbt cat future will lost
2,0,madam read good book
3,0,thuuu samuel new international take niv this i...
4,0,guys dont fall on her talk this is kali yuga s...
...,...,...
4525,0,this wont suits for india it is against to the...
4526,0,human being going lpgt this is very bull give ...
4527,0,more are momosexual affected with std
4528,1,Today only I watch this .....actually it is bi...


In [ ]:
df_aug_shuffled['label'].value_counts()

0    2826
1    1500
2     204
Name: label, dtype: int64

# 4. Save non-preprocessed but augmented data
  *   Note : this   only makes sense if you have not run cells 2.7 through 2.10

In [ ]:
df_aug_shuffled.to_csv('eng_train_nonprepro_aug.csv', index=False)
df_eng_dev.to_csv('eng_dev_nonprepro_aug.csv', index=False)
df_eng_test.to_csv('eng_test_nonprepro_aug.csv', index=False)

# 5. Saving data

**Save augmented english data**

In [ ]:
df_aug_shuffled.to_csv('eng_augmented_data.csv', index=False)

In [ ]:
df_eng_dev.to_csv('eng_dev.csv', index=False)

In [ ]:
df_eng_test.to_csv('eng_test.csv', index=False)

**Save english data without augmentation**

In [ ]:
df_eng_train.to_csv('eng_train.csv', index=False)

**Save tamil data**

In [ ]:
df_tam_train.to_csv('tam_augmented_data.csv', index=False)

In [ ]:
df_tam_dev.to_csv('tam_dev.csv', index=False)

In [ ]:
df_tam_test.to_csv('tam_test.csv', index=False)

**Save code-mixed data**

In [ ]:
df_eng_tam_train.to_csv('eng_tam_augmented_data.csv', index=False)

In [ ]:
df_eng_tam_dev.to_csv('eng_tam_dev.csv', index=False)

In [ ]:
df_eng_tam_test.to_csv('eng_tam_test.csv', index=False)